In [ ]:
from utils.utils import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils.LSTM import *
import yaml

In [2]:
# pip install optuna
import optuna
import numpy as np
import pandas as pd

# ==== Prepara data (leída una sola vez) ====
stocks = ["MSFT", "INTU", "AMD", "AAPL"]
dfs = {s: pd.read_csv(f"./prices/{s}_prices.csv", index_col=0, parse_dates=True) for s in stocks}

start_day      = "2025-01-01"
horizon_days   = 50           # días hábiles a predecir
training_years = 2
feat_cols      = ["Close"]
W              = 50
hidden_size    = 64
num_layers     = 2
patience       = 10
torch_seed     = 0
loss_validation = True
hit_coef       = 0.01

def eval_mse(res):
    res_df = res[0]
    return float(np.mean((res_df["y_pred_norm"] - res_df["y_true_norm"])**2))

# ==== Función objetivo de Optuna ====
def objective(trial):
    # espacio de búsqueda
    bs      = trial.suggest_categorical("batch_size", [5, 8, 11, 16, 21, 32])
    lr      = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.6)

    mses = []
    for stock, df in dfs.items():
        res = walk_forward_predict(
            LSTMRegressor,
            df,
            start_day,
            horizon_days,
            training_years=training_years,
            feat_cols=feat_cols,
            W=W,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            hit_coef=hit_coef,
            batch_size=bs,
            epochs=30,               # podés subirlo si te da el tiempo
            lr=lr,
            patience=patience,
            loss_validation=loss_validation,
            torch_seed=torch_seed
        )
        mses.append(eval_mse(res))

    # promedio entre acciones (podés cambiar a mediana si hay outliers)
    return float(np.mean(mses))

# ==== Ejecuta la optimización ====
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Mejores hiperparámetros:", study.best_params)
print("MSE promedio (best):", study.best_value)

# ==== Re-entrena con los mejores hiperparámetros y devuelve results ====
best_bs      = study.best_params["batch_size"]
best_lr      = study.best_params["lr"]
best_dropout = study.best_params["dropout"]

results = []
for stock, df in dfs.items():
    res = walk_forward_predict(
        LSTMRegressor,
        df,
        start_day,
        horizon_days,
        training_years=training_years,
        feat_cols=feat_cols,
        W=W,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=best_dropout,
        hit_coef=hit_coef,
        batch_size=best_bs,
        epochs=30,
        lr=best_lr,
        patience=patience,
        loss_validation=loss_validation,
        torch_seed=torch_seed
    )
    results.append({"stock": stock, "res": res})

# Opcional: ver el score por stock con los hiperparámetros óptimos
for item in results:
    
    stock, res = item["stock"], item["res"]
    print(stock, "MSE =", eval_mse(res))


c:\dev\MIA\3 er Trimestre\Aprendizaje Automático II\stock-hybrid-forecasting\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-04 18:06:56,262] A new study created in memory with name: no-name-1d7a9d7e-8d27-4111-84ec-f6a450282275
Best trial: 0. Best value: 0.00531982:   2%|▎         | 1/40 [11:34<7:31:22, 694.42s/it]

[I 2025-11-04 18:18:30,682] Trial 0 finished with value: 0.005319818579343579 and parameters: {'batch_size': 11, 'lr': 0.00019151567256973535, 'dropout': 0.3430368848743234}. Best is trial 0 with value: 0.005319818579343579.


Best trial: 1. Best value: 0.0032746:   5%|▌         | 2/40 [19:53<6:07:01, 579.52s/it] 

[I 2025-11-04 18:26:49,771] Trial 1 finished with value: 0.0032745984868505886 and parameters: {'batch_size': 16, 'lr': 0.0007600553592348185, 'dropout': 0.3760226501926817}. Best is trial 1 with value: 0.0032745984868505886.


Best trial: 2. Best value: 0.00143938:   8%|▊         | 3/40 [24:21<4:29:31, 437.06s/it]

...

[I 2025-11-05 04:48:37,094] Trial 38 finished with value: 0.0030703891178463483 and parameters: {'batch_size': 32, 'lr': 0.0008866746452792342, 'dropout': 0.21066196485074346}. Best is trial 22 with value: 0.0010536778916178675.


Best trial: 22. Best value: 0.00105368: 100%|██████████| 40/40 [11:10:19<00:00, 1005.49s/it]


[I 2025-11-05 05:17:15,969] Trial 39 finished with value: 0.0034029423603858682 and parameters: {'batch_size': 5, 'lr': 0.0003453728857764511, 'dropout': 0.3266083483223248}. Best is trial 22 with value: 0.0010536778916178675.
Mejores hiperparámetros: {'batch_size': 8, 'lr': 0.0031178165012961004, 'dropout': 0.005614091359169214}
MSE promedio (best): 0.0010536778916178675


Processing test days: 100%|██████████| 50/50 [04:52<00:00,  5.85s/it]

MSFT MSE = 0.000979578818431599
INTU MSE = 0.0016765967357358622
AMD MSE = 0.00044318569003377564
AAPL MSE = 0.0011153503222702333


In [3]:
best_bs    

8

In [ ]:
best_lr      


0.0031178165012961004

In [5]:
best_dropout

0.005614091359169214